<img src="https://raw.githubusercontent.com/ryanedw/COMPSS-202-SU24/main/Images/UCB-macss.jpg" width="120" align="right"/>
<h1>COMPSS 202 Class 08</h1>

<h2>Errors in Regression</h2>

Inspired by [SticiGui Chapter 11](https://www.stat.berkeley.edu/~stark/SticiGui/Text/regressionErrors.htm)

<h3>Learning objectives:</h3>

<ol style="margin-top: 0; margin-bottom: 0;">
  <li>More about regression
   <ul style="margin-top: 0; margin-bottom: 0;">
      <li>Etymology: regression is the opposite of progression
      </li>
      <li>A regression model yields a great prediction of $Y$ given $X$, but the prediction is hardwired to show regression to the mean
      </li>
      <li>The regression fallacy is to ignore this hardwiring and instead attribute regression to the mean to a “jinx” or a third variable like praise or criticism
      </li>
   </ul>
  </li>
      <li>Residuals, sum of squared errors, $R^2 = r \times r$, and overall regression “fit”
   <ul style="margin-top: 0; margin-bottom: 0;">
      <li>(This is not the same as statistical significance, which we’re building up to)
      </li>
    </ul>
   </li>
</ol>





To begin, please run the cells below to load up the libraries necessary to access data in Google Sheets. Best practices include running the cells in order.

In [ ]:
install.packages("googlesheets4")
library(googlesheets4)
gs4_deauth()

<h2>1. Review of earlier results with the Pearson heights data</h2>

Here again are 1,078 observations of "fathers" and "sons" from a well-known training dataset based on the historical work of [Karl Pearson](https://en.wikipedia.org/wiki/Karl_Pearson). Please see the Class 03 notebook for more details.

Here is a direct link to the Google Sheets file loaded in the cell below: [Pearson height data.sheets](https://docs.google.com/spreadsheets/d/1TZhFGjT-uXd9ScucSYkT0MNARNDMCRCbAQgx4jac-X8/edit?usp=drive_link)

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1TZhFGjT-uXd9ScucSYkT0MNARNDMCRCbAQgx4jac-X8/edit?usp=drive_link"

pheights <- read_sheet(sheet_url,
                       range = "B13:D1091")

Calling `head()` provides a useful quick look at the top of the dataset. Calling `dim()` helps us make sure we have the right dataset loaded up in the correct way.

In [ ]:
head(pheights)
dim(pheights)

Calculating the Pearson correlation cleanly seems to require passing a few options. `method = "pearson"` appears to be redundant here, but I'll include it anyway. __R__ and other statistical programs tend to get finicky about missing observations, and `use = "complete.obs"` seems to help.

In [ ]:
r = cor(pheights$father, pheights$son,
        method = "pearson",
        use = "complete.obs"
       )
r

As we discussed in the notebook for Class 06, it turns out that we can also use `lm()` to estimate the blue line using <b>ordinary least squares</b>, which we will return to later in COMPSS 202.

The syntax of `lm()` is as follows, where the funny part with the tilde (~) is the estimation equation, with a tilde instead of an equals sign and no coefficients formally listed:

In [ ]:
reg1 <- lm(son ~ father,
          data = pheights)
summary(reg1)

Notice how `lm()` produces a `Multiple R-squared:` toward the bottom of the summary window. For a bivariate regression with one $Y$ regressed on one $X$, this is simply the square of the Pearson correlation, $r$. To wit: 

In [ ]:
r^2

<h2>2. Some remarks about $R^2$</h2>

<h3>2.1. What the $R^2$ is</h3>
As shown in the Class 08 slides, the $R^2$ <b>measures the total amount of variation in $Y$ that the model can explain</b> with a constant term and with the variance in $X$. How can we show this? What's the total variance in $Y$?
$$
Var(Y) = \frac{1}{n} \sum_n (Y - \bar{Y})^2
$$
And the math in the slides shows this:
$$
R^2 = \frac{\sum_n (\hat{Y} - \bar{Y})^2}
{\sum_n (Y - \bar{Y})^2}
$$
The denominator is $n$ times the variance in $Y$. The numerator is $n$ times the variance in the OLS prediction $\hat{Y}$. The $n$'s cancel, and the formula is thus shorthand for the ratio of the total variance in the OLS prediction as a share of the total variance in the outcome variable.

<h3>2.2. What the $R^2$ is NOT</h3>

We currently have <b>nothing to say about the "significance" of $X$</b> or anything else in predicting $Y$. All the $R^2$ tells us about is the fit of the model.

Because there are several moving pieces here, the model fit is really all we can discuss at this stage, even though we would probably like to say something about the relationship between father's height $X$ and son's height $Y$.

<h3>2.3. Is $R^2 = 0.25$ too small, too big, or just right?</h3>

Goldilocks would clearly choose to run Baby Bear's model, which fit just right!

Model fit is a funny thing in statistics generally, and I think it's especially quirky in social science, where usually we care a lot more about the statistical significance and causality of a treatment effect on an outcome, and we care less about the model fit per se. A treatment effect is a parameter like $\beta$, if the $X$ variable represents something like receiving a medical treatment, taking a drug, receiving a cash transfer, or winning a charter school lottery.

The outcome variables ($Y$) that we usually care about in social science are things like years of schooling or diplomas attained, earnings, self-reported happiness, good health, and so on. Each of these $Y$'s are themselves shifted by many other variables that are often outside the reach of policy levers, like our childhood conditions, our parents, our genes, systemic racism and discrimination, and so on. Thus it would be natural to find $R^2$'s that are pretty low, because our outcome variables are shifted by many other sources of variation in addition, hopefully, to the policy variable of interest.

An $R^2 = 0.25$ in some circles might cause gasps of joy, while in other circles may be cause for concern. It depends a lot on the objective of the modeling. If it's critical to predict $Y$ with low error, then $0.25$ isn't great. If instead we care about the ability to <b>reject the null hypothesis</b> that $\beta = 0$, then ...

<div style="text-align: right">... stay tuned for the second half of COMPSS 202!</div>


---

<h2>3. Regression to the mean in flights data</h2>

As the slides lay out, the <b>regression fallacy</b> is the mistaken belief that something other than "regression to the mean" is driving the likelihood of average results after either above-average or below-average results. It's the false belief in a causal meaning behind the statistical likelihood of mediocre peformance following either excellent or poor performance.

The classic paper in this field is [Tversky and Kahneman (1974)](https://www-jstor-org.libproxy.berkeley.edu/stable/1738360), who examined flight training data in Israel and found:
* Strong regression to the mean in the quality of a pilot's successive flights
* Strong beliefs that praise was harmful and criticism was valuable

The two of these appearing together is not an accident; it is the regression fallacy in operation. Pilots who really "stuck the landing" (i.e., did it very well, above average) might receive praise. Pilots who were waved off might receive criticism. If regression to the mean is strong in flight training, pilots who did very well and incidentally received praise would likely do much more average on the next flight, regressing to the mean. Pilots who did very poorly and received criticism would likely do much more average on the next flight, regressing to the mean.

There could easily be no true causal effect of praise or criticism in such a context, but they would be correlated because of the regression effect, and observers would and did conclude that praise was harmful and criticism was good. This formed a key part of Daniel Kahneman's Nobel oeuvre.

<h3>3.1. Flights data from Dorsey-Palmateer and Smith (2006)</h3>

[Dorsey-Palmateer and Smith (2006)](https://economics-files.pomona.edu/garysmith/papers/flightTests.pdf) find strong regression to the mean in U.S. Navy flight training data, which they summarize in their Table 3, reprinted below:

<img src="https://raw.githubusercontent.com/ryanedw/COMPSS-202-SU24/main/Images/dorsey-palmateer-smith-table3.png" width="500" align="center"/>

These are subjective scores with qualitative labels that are described in the paper and in the slides for Class 08. The important thing is that 1 is the worst and 5 is the best. There are clear modes at score 3.

I created a dataset with 1,711 observations of two variables: `previous` and `current` contained in the Google Sheets file below. As shown in the slides for Class 08, I had also years ago created two perturbed versions of these data `pp` and `cp` using the uniform random number generator in Microsoft Excel and simulating two additive errors distributed $N(0,0.5^2)$, i.e. drawn from a normal distribution with mean $0$ and standard deviation $0.5$.

Here is a direct link to the Google Sheets file loaded in the cell below: [flights.sheets](https://docs.google.com/spreadsheets/d/1wkZQc8PWq-J0NS8-oFAHGaWqctFTnB_jntgn6K8M-XA/edit?usp=sharing)

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1wkZQc8PWq-J0NS8-oFAHGaWqctFTnB_jntgn6K8M-XA/edit?usp=sharing"

flights <- read_sheet(sheet_url,
                      range = "A1:E1712")

In [ ]:
head(flights)
mean(flights$previous)
mean(flights$current)

Below is a look at the unadjusted data, which produce a pretty indecipherable scatterplot because are discrete in nature.

In [ ]:
plot(flights$previous, flights$current)

<h3>3.2. Visualizations like that are unfortunately common</h3>

In social science, it is common for outcome and treatment variables to be lumpy and discrete rather than continuous. The original dataset of heights collected by Pearson and Galton, for example, was much more like this in nature than it was like the training version we have been examining. In cases like these, scatterplots might just create more confusion rather than help us visualize relationships.

It turns out that __R__ has a built-in function `jitter()`, [documented here](https://search.r-project.org/R/refmans/base/html/jitter.html) which can inject randomness into observations. Below is some code that does it, with the `amount` parameter set to 1. It's good form to `set.seed()` to something, perhaps today's date, so that you can reproduce what you see.

In [ ]:
set.seed(20240612)

plot(jitter(flights$previous, amount = 1), 
     jitter(flights$current, amount = 1))

The documentation reports that `jitter()` is using uniformly distributed random variables. I think normally distributed random variables are likely to look better. Here is some code that resets the seed and calls `rnorm()` with mean 0 and variance equal to `randvar`, then it adds those random shocks onto the data:

In [ ]:
set.seed(20240612)
randvar = 0.36
flights$rand1 = rnorm(nrow(flights),0,randvar)
flights$rand2 = rnorm(nrow(flights),0,randvar)

flights$previousr = flights$previous + flights$rand1
flights$currentr  = flights$current + flights$rand2

# Uncomment these as needed for diagnostic purposes 
#head(flights)
#sd(flights$rand1)
#sd(flights$rand2)

Let's look at the visualization with my normally distributed shocks. This should look more like the image in the slides, which is what I originally did with Excel some years ago. (Those data are included as `pp` and `cp` if you're interested in examining them here.) 

In [ ]:
plot(flights$previousr, flights$currentr)

Definitely better, but of course a challenge is that we're trying to reshape something that is basically a nebulous cloud, not an obvious football.

Here is the least squares regression run on these data that I perturbed by hand. We are estimating this equation:

$$
current_i = \alpha + \beta \ previous_i + \epsilon_i
$$

where $previous_i$ is the quality of the pilot's previous flight and $current_i$ is the quality of the current flight. As usual, $\epsilon_i$ is a white-noise error.

In [ ]:
reg_flightsr <- lm(currentr ~ previousr,
                   data = flights
                   )
summary(reg_flightsr)

---

In the randomly perturbed data, we see estimates of $\hat{\alpha} = 2.2$ and $\hat{\beta} = 0.17$. 

And now for comparison, here is `lm()` run with the uncorrected data:

In [ ]:
reg_flights <- lm(current ~ previous,
                  data = flights
                  )
summary(reg_flights)

---

In the actual data, $\hat{\beta} = 0.21$, which is larger than what we see in the randomly perturbed data. This happens because we have injected classical <i>measurement error</i> into $X$ as well as $Y$. Measurement error in $Y$ can be modeled as part of the white-noise term $\epsilon$, but measurement error in $X$ leads to <i>attenuation</i> of regression coefficients: they become smaller in magnitude. 

<h3>3.3. Bottom line: strong regression to the mean</h3>

A main punchline here is that in U.S. Navy data, there is strong regression to the mean. It is true that $\hat{\beta} = 0.2$ or so, and as we will see in the second half of COMPSS 202, that coefficient is statistically significant and thus we can reject the null hypothesis that $\beta = 0$. But $\beta = 0.2$ is small enough to mean that a pilot who scores $previous = X = 4$ is likely to score 

$$
current = \hat{Y} = \hat{\alpha} + \hat{\beta} \times 4 = 2 + 0.2 \times 4 = 2 + 0.8 = 2.8
$$ 

on the next round. That's pretty close to the average, $2.6$.

Under conditions such as these, it seems likely a casual observer would mistake strong regressin to the mean with purported effects of praising or criticizing, even if those effects did not exist. 

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>